### Scraping metadata from Snoopsnoo.com on users' favorite subreddits and topics of interests 

This data will be used to build a Flask app that recommends subreddits & topics based on the user's MBTI type

In [3]:
from __future__ import print_function, division

In [2]:
# if needed: pip install requests
from bs4 import BeautifulSoup
import requests
import re
import json
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
from random import randint


In [12]:
all_cases = pd.read_csv('9_6_clean_labels.csv')

In [13]:
all_cases = all_cases.drop_duplicates(['user_orig'])

In [14]:
all_cases.head(1)

,Unnamed: 0.1.1,user,comment,user_orig,comment_orig,personality_label
0,59a7564941912db5409558c7,_____-______________,Also include in your response if you have SCT ...,b'_____-______________',"b""Also include in your response if you have SC...",INTP


**list for users**

In [ ]:
#get a list of all usernames from existing data to create url links
usr_get = [str(i) for i in all_cases.user]

In [71]:
usr_get

['SempiSenpai',
 'Getjac',
 'GeoPizza',
 'paperclip1213',
 'metao',
 'JambeardReborn',
 'Cephatyl',
 'OncleJulian',
 'UncleFab',
 'walkthroughthefire',
 'Doomhammer458',
 'mcr55',
 'App1eEater',
 'HagalUlfr',
 'Semtec',
 'mindlessmusings',
 'iJoelPeterPage',
 'TerWood',
 'TheBenno',
 'feldsparred',
 'loveandmayhem',
 'literallyhere',
 'DownInDemerara',
 'imbaZarkout',
 'tvxcute',
 'theeburr',
 'vldmort',
 'unwordableweirdness',
 'introvertedintooit',
 '0hfuck',
 'WhenIDipUDipWeDip',
 'goNe-Deep',
 'Pit9',
 'Kubiben',
 'PrincessNips',
 'Kenwric',
 'best4bond',
 'anybodyanywhere',
 'iTo',
 'PM_ME_YOUR_BURDENS',
 'NotsowiseSAGe',
 'DemiPixel',
 'dinsette',
 'thecarebearcares',
 'chair_sitting',
 '_mrfoobar',
 'mscarsonelle',
 'jagdbogentag',
 'Gary_Oldman_AMA',
 'ssarahhhh22',
 'frenzyboard',
 'JCY2K',
 'Twentey',
 'Logiq_',
 'stinkyrossignol',
 'Comyu',
 'Qaz_',
 'TrappinOutTheZoo',
 '130alexandert',
 'Allanon_2020',
 'rosechiffon',
 'MartholomewMind',
 'miralsad',
 'Schmucko',
 'hutacar

**Scrape Snoopsnu.com for meta data**

In [79]:
def get_urls(user_list):
    '''Adds reddit username to snoosnoo url address'''
    url_prefix = 'https://snoopsnoo.com/u/'
    new_url=[]
    for user in user_list: 
        new_url.append(url_prefix+user)
    return new_url

In [80]:
url_list = get_urls(usr_get)
url_list[0:10]

['https://snoopsnoo.com/u/6-Oct',
 'https://snoopsnoo.com/u/1.15E+13',
 'https://snoopsnoo.com/u/_____non',
 'https://snoopsnoo.com/u/_-ThrownAway-_',
 'https://snoopsnoo.com/u/_akay',
 'https://snoopsnoo.com/u/_bombs',
 'https://snoopsnoo.com/u/_cheerio',
 'https://snoopsnoo.com/u/_chefboyarlee_',
 'https://snoopsnoo.com/u/_DeceivingFox',
 'https://snoopsnoo.com/u/_Der_Hammer_']

In [81]:
snoopsnu = {}

In [82]:
karma = []
username = []
subreddits = []
topics = []

In [83]:
#scrapes Snoopsnu for for data from each user
for url in url_list:
    time.sleep(randint(0,2))
    try:
        response = requests.get(url.strip())
        page = response.text
        soup = BeautifulSoup(page,"lxml")
    except: 
        continue
    for element in soup.find_all('script', text = re.compile('{"name": (.*), "size":')):
        try:
            json_readable = json.loads(str(element).split('stringify(')[1].split(');')[0])
            username.append(json_readable['username'])
            karma.append(json_readable['summary']['comments']['average_karma'])
            sub_list = []
            for sub_red in json_readable['summary']['submissions']['type_domain_breakdown']['children'][0]['children']:
                sub_list.append(sub_red['name'])
            subreddits.append(sub_list)
            topic_list = []
            for topic in json_readable['metrics']['topic']['children']:
                topic_list.append(topic['name'])
            topics.append(topic_list)
            snoopsnu['username'] = username
            snoopsnu['karma'] = karma
            snoopsnu['subreddits'] = subreddits
            snoopsnu['topics'] = topics
        except:
            continue

In [ ]:
data = pd.DataFrame(snoopsnu)

In [49]:
data.head()

,Unnamed: 0,Unnamed: 0.1,karma,subreddits,topics,username
0,0,NaN,2.0,"['C_S_T', 'ENFP', 'DotA2']","['Other', 'Lifestyle', 'Gaming', 'News and Pol...",SempiSenpai
1,1,NaN,10.0,"['SomebodyMakeThis', 'CampAndHikeMichigan', 't...","['General', 'Entertainment', 'Gaming', 'Hobbie...",Getjac
2,2,NaN,2.0,['fireemblem'],"['Gaming', 'Other', 'General']",GeoPizza
3,3,NaN,8.0,"['unitedkingdom', 'ShittyFanTheories', 'self',...","['General', 'Lifestyle', 'Locations', 'Adult a...",paperclip1213
4,4,NaN,4.0,"['perth', 'CookieClicker', 'FanTheories', 'Wil...","['Locations', 'Lifestyle', 'Entertainment', 'G...",metao


In [58]:
data.drop(['Unnamed: 0','Unnamed: 0.1.1', 'comment', 'comment_orig'], inplace=True, axis=1)

In [ ]:
data.to_csv('snoopsnu.csv')